#### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 不平衡数据的分类

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://tensorflow.google.cn/tutorials/structured_data/imbalanced_data" class=""><img src="https://tensorflow.google.cn/images/tf_logo_32px.png" class="">在 TensorFlow.org 上查看</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/structured_data/imbalanced_data.ipynb" class=""><img src="https://tensorflow.google.cn/images/colab_logo_32px.png" class="">在 Google Colab 中运行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/structured_data/imbalanced_data.ipynb" class=""><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png" class="">在 GitHub 上查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/structured_data/imbalanced_data.ipynb" class=""><img src="https://tensorflow.google.cn/images/download_logo_32px.png" class="">下载笔记本</a></td>
</table>

本教程演示了如何对高度不平衡的数据集进行分类，在此类数据集中，一类中的样本数量远多于另一类中的样本数量。您将使用 Kaggle 上托管的 [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud) 数据集，目的是从总共 284,807 笔交易中检测出仅有的 492 笔欺诈交易。您将使用 [Keras](../../guide/keras/overview.ipynb) 来定义模型和[类权重](https://tensorflow.google.cn/versions/r2.0/api_docs/python/tf/keras/Model)，从而帮助模型从不平衡数据中学习。

本教程包含下列操作的完整代码：

- 使用 Pandas 加载 CSV 文件。
- 创建训练、验证和测试集。
- 使用 Keras 定义并训练模型（包括设置类权重）。
- 使用各种指标（包括精确率和召回率）评估模型。
- 尝试使用常见技术来处理不平衡数据，例如：
    - 类加权
    - 过采样


## 设置

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## 数据处理与浏览

### 下载 Kaggle Credit Card Fraud 数据集

Pandas 是一个 Python 库，其中包含许多有用的实用工具，用于加载和使用结构化数据，并可用于将 CSV 下载到数据帧中。

注：Worldline 和 ULB（布鲁塞尔自由大学）[机器学习小组](http://mlg.ulb.ac.be)在大数据挖掘和欺诈检测的合作研究期间，已对此数据集进行了收集和分析。与相关主题当前和过去项目有关的详细信息，请访问[这里](https://www.researchgate.net/project/Fraud-detection-5)和 [DefeatFraud](https://mlg.ulb.ac.be/wordpress/portfolio_page/defeatfraud-assessment-and-validation-of-deep-feature-engineering-and-learning-solutions-for-fraud-detection/) 项目页面。

In [ ]:
file = tf.keras.utils
raw_df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
raw_df.head()

In [ ]:
raw_df[['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V26', 'V27', 'V28', 'Amount', 'Class']].describe()

### 检查类标签的不平衡

让我们看一下数据集的不平衡情况：

In [ ]:
neg, pos = np.bincount(raw_df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

这表明正样本的比例很小。

### 清理、拆分和归一化数据

原始数据有一些问题。首先，`Time` 和 `Amount` 列变化太大，无法直接使用。删除 `Time` 列（因为不清楚其含义），并获取 `Amount` 列的日志以缩小其范围。

In [ ]:
cleaned_df = raw_df.copy()

# You don't want the `Time` column.
cleaned_df.pop('Time')

# The `Amount` column covers a huge range. Convert to log-space.
eps=0.001 # 0 =&gt; 0.1¢
cleaned_df['Log Ammount'] = np.log(cleaned_df.pop('Amount')+eps)

将数据集拆分为训练、验证和测试集。验证集在模型拟合期间使用，用于评估损失和任何指标，判断模型与数据的拟合程度。测试集在训练阶段完全不使用，仅在最后用于评估模型泛化到新数据的能力。这对于不平衡的数据集尤为重要，因为[过拟合](https://developers.google.com/machine-learning/crash-course/generalization/peril-of-overfitting)是缺乏训练数据造成的一个重大问题。

In [ ]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

使用 sklearn StandardScaler 将输入特征归一化。这会将平均值设置为 0，标准偏差设置为 1。

注：`StandardScaler` 只能使用 `train_features` 进行拟合，以确保模型不会窥视验证集或测试集。 

In [ ]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)


小心：如果要部署模型，保留预处理计算至关重要。这是将它们实现为层并在导出之前将它们附加到模型最简单的方法。


### 查看数据分布

接下来通过一些特征比较一下正样本和负样本的分布。此时，建议您问自己如下问题：

- 这些分布是否有意义？
    - 是。您已对输入进行了归一化处理，而它们大多集中在 `+/- 2` 范围内。
- 您是否能看出分布之间的差异？
    - 是。正样本包含极值的比率高得多 。

In [ ]:
pos_df = pd.DataFrame(train_features[ bool_train_labels], columns = train_df.columns)
neg_df = pd.DataFrame(train_features[~bool_train_labels], columns = train_df.columns)

sns.jointplot(pos_df['V5'], pos_df['V6'],
              kind='hex', xlim = (-5,5), ylim = (-5,5))
plt.suptitle("Positive distribution")

sns.jointplot(neg_df['V5'], neg_df['V6'],
              kind='hex', xlim = (-5,5), ylim = (-5,5))
_ = plt.suptitle("Negative distribution")

## 定义模型和指标

定义一个函数，该函数会创建一个简单的神经网络，其中包含一个密集连接的隐藏层、一个用于减少过拟合的[随机失活](https://developers.google.com/machine-learning/glossary/#dropout_regularization)层，以及一个返回欺诈交易概率的输出 Sigmoid 层： 

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

def make_model(metrics = METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(lr=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

### 了解有用的指标

请注意，上面定义的一些指标可以由模型计算得出，这对评估性能很有帮助。

- **假**负例和**假**正例是被**错误**分类的样本
- **真**负例和**真**正例是被**正确**分类的样本
- **准确率**是被正确分类的样本的百分比

> $\frac{\text{true samples}}{\text{total samples}}$

- **精确率**是被正确分类的**预测**正例的百分比

> $\frac{\text{true positives}}{\text{true positives + false positives}}$

- **召回率**是被正确分类的**实际**正例的百分比

> $\frac{\text{true positives}}{\text{true positives + false negatives}}$

- **AUC** 是指接收器操作特征曲线中的曲线下方面积 (ROC-AUC)。此指标等于分类器对随机正样本的排序高于随机负样本的概率。

注：准确率在此任务中不是一个有用的指标。只要始终预测“False”，您就可以在此任务中达到 99.8%+ 的准确率。

延伸阅读：

- [真与假以及正类别与负类别](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative)
- [准确率](https://developers.google.com/machine-learning/crash-course/classification/accuracy)
- [精确率和召回率](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall)
- [ROC-AUC](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)

## 基线模型

### 构建模型

现在，使用先前定义的函数创建并训练模型。请注意，该模型使用大于默认的批次大小 (2048) 来进行拟合，这一点很重要，有助于确保每个批次都有一定机会包含少量正样本。如果批次过小，它们可能会没有可供学习的欺诈交易。

注：此模型无法很好地处理类不平衡问题。我们将在本教程的后面部分对此进行改进。

In [ ]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [ ]:
model = make_model()
model.summary()

试运行模型：

In [ ]:
model.predict(train_features[:10])

### 可选：设置正确的初始偏差。

模型最初的猜测不太理想。您知道数据集不平衡，因此需要设置输出层的偏差以反映这种不平衡（请参阅：[训练神经网络的秘诀：“好好初始化”](http://karpathy.github.io/2019/04/25/recipe/#2-set-up-the-end-to-end-trainingevaluation-skeleton--get-dumb-baselines)）。这样做有助于初始收敛。

使用默认偏差初始化时，损失应约为 `math.log(2) = 0.69314`。 

In [ ]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

可以用以下代码推导出要设置的正确偏差：

$$ p_0 = pos/(pos + neg) = 1/(1+e^{-b_0}) $$ $$ b_0 = -log_e(1/p_0 - 1) $$ $$ b_0 = log_e(pos/neg)$$

In [ ]:
initial_bias = np.log([pos/neg])
initial_bias

将其设置为初始偏差，模型将给出合理得多的初始猜测。

结果应该接近：`pos/total = 0.0018`

In [ ]:
model = make_model(output_bias = initial_bias)
model.predict(train_features[:10])

使用此初始化，初始损失应约为：

$$-p_0log(p_0)-(1-p_0)log(1-p_0) = 0.01317$$

In [ ]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

此初始损失大约是使用朴素初始化时损失的 50 倍。

这样，模型就不需要花费前几个周期去仅仅了解不可能有正样本。这也使得在训练过程中更容易读取损失图。

### 为初始权重设置检查点

要使各种训练运行更具可比性，请将此初始模型的权重保存在检查点文件中，并在训练之前将其加载到每个模型中。

In [ ]:
initial_weights = os.path.join(tempfile.mkdtemp(),'initial_weights')
model.save_weights(initial_weights)

### 确认偏差修正有帮助

在继续之前，迅速确认这一细致偏差初始化是否确实起了作用。

在使用和不使用此细致初始化的情况下，将模型训练 20 个周期，并比较损失： 

In [ ]:
model = make_model()
model.load_weights(initial_weights)
model.layers[-1].bias.assign([0.0])
zero_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [ ]:
model = make_model()
model.load_weights(initial_weights)
careful_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [ ]:
def plot_loss(history, label, n):
  # Use a log scale to show the wide range of values.
  plt.semilogy(history.epoch,  history.history['loss'],
               color=colors[n], label='Train '+label)
  plt.semilogy(history.epoch,  history.history['val_loss'],
          color=colors[n], label='Val '+label,
          linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  
  plt.legend()

In [ ]:
plot_loss(zero_bias_history, "Zero Bias", 0)
plot_loss(careful_bias_history, "Careful Bias", 1)

上图清楚表明：就验证损失而言，在这个问题上，此细致初始化具有明显优势。 

### 训练模型

In [ ]:
model = make_model()
model.load_weights(initial_weights)
baseline_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_features, val_labels))

### 查看训练历史记录

在本部分，您将针对训练集和验证集生成模型的准确率和损失绘图。这些对于检查过拟合十分有用，您可以在此[教程](https://tensorflow.google.cn/tutorials/keras/overfit_and_underfit)中了解更多信息。

此外，您还可以为您在上面创建的任何指标生成上述绘图。假负例包含在以下示例中。

In [ ]:
def plot_metrics(history):
  metrics =  ['loss', 'auc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()


In [ ]:
plot_metrics(baseline_history)

注：验证曲线通常比训练曲线表现更好。这主要是由于在评估模型时，随机失活层处于非活动状态。

### 评估指标

您可以使用[混淆矩阵](https://developers.google.com/machine-learning/glossary/#confusion_matrix)来汇总实际标签和预测标签，其中 X 轴为预测标签，Y 轴为实际标签。

In [ ]:
train_predictions_baseline = model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_baseline = model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

在测试数据集上评估模型，并根据您在上面创建的指标显示结果。

In [ ]:
baseline_results = model.evaluate(test_features, test_labels,
                                  batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_baseline)

如果模型完美地预测了所有内容，则这是一个[对角矩阵](https://en.wikipedia.org/wiki/Diagonal_matrix)，其中偏离主对角线的值（表示不正确的预测）将为零。在这种情况下，矩阵会显示您的假正例相对较少，这意味着被错误标记的合法交易相对较少。但是，您可能希望得到更少的假负例，即使这会增加假正例的数量。这种权衡可能更加可取，因为假负例允许进行欺诈交易，而假正例可能导致向客户发送电子邮件，要求他们验证自己的信用卡活动。

### 绘制 ROC

现在绘制 [ROC](https://developers.google.com/machine-learning/glossary#ROC)。此绘图非常有用，因为它一目了然地显示了模型只需通过调整输出阈值就能达到的性能范围。

In [ ]:
def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  plt.xlim([-0.5,20])
  plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

看起来精确率相对较高，但是召回率和 ROC 曲线下方面积 (AUC) 可能并没有您期望的那么高。当试图同时最大限度地提高精确率和召回率时，分类器通常会面临挑战，在处理不平衡数据集时尤其如此。请务必根据您所关心的问题来考虑不同类型错误的代价。在此示例中，假负例（漏掉欺诈交易）可能造成财务损失，而假正例（将交易错误地标记为欺诈）则可能降低用户满意度。

## 类权重

### 计算类权重

我们的目标是识别欺诈交易，但您没有很多可以使用的此类正样本，因此您希望分类器提高可用的少数样本的权重。为此，您可以使用参数将 Keras 权重传递给每个类。这些权重将使模型“更加关注”来自代表不足的类的样本。

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

### 使用类权重训练模型

现在，尝试使用类权重对模型进行重新训练和评估，以了解其对预测的影响。

注：使用 `class_weights` 会改变损失范围。这可能会影响训练的稳定性，具体取决于优化器。步长取决于梯度大小的优化器（如 `optimizers.SGD`）可能会失效。此处使用的优化器（`optimizers.Adam`）不受缩放更改的影响。还要注意，由于加权，两个模型之间的总损失不具可比性。

In [ ]:
weighted_model = make_model()
weighted_model.load_weights(initial_weights)

weighted_history = weighted_model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_features, val_labels),
    # The class weights go here
    class_weight=class_weight) 

### 查看训练历史记录

In [ ]:
plot_metrics(weighted_history)

### 评估指标

In [ ]:
train_predictions_weighted = weighted_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_weighted = weighted_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
weighted_results = weighted_model.evaluate(test_features, test_labels,
                                           batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(weighted_model.metrics_names, weighted_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_weighted)

在这里，您可以看到，使用类权重时，由于存在更多假正例，准确率和精确率较低，但是相反，由于模型也找到了更多真正例，召回率和 AUC 较高。尽管准确率较低，但是此模型具有较高的召回率（且识别出了更多欺诈交易）。当然，两种类型的错误都有代价（您也不希望因将过多合法交易标记为欺诈来打扰客户）。请在应用时认真权衡这些不同类型的错误。

### 绘制 ROC

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')


plt.legend(loc='lower right')

## 过采样

### 对占少数的类进行过采样

一种相关方法是通过对占少数的类进行过采样来对数据集进行重新采样。

In [ ]:
pos_features = train_features[bool_train_labels]
neg_features = train_features[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

#### 使用 NumPy

您可以通过从正样本中选择正确数量的随机索引来手动平衡数据集：

In [ ]:
ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

res_pos_features.shape

In [ ]:
resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape

#### 使用 `tf.data`

如果您使用的是 `tf.data`，则生成平衡样本最简单的方法是从 `positive` 和 `negative` 数据集开始，然后将它们合并。有关更多示例，请参阅 [tf.data 指南](../../guide/data.ipynb)。

In [ ]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)

每个数据集都会提供 `(feature, label)` 对：

In [ ]:
for features, label in pos_ds.take(1):
  print("Features:\n", features.numpy())
  print()
  print("Label: ", label.numpy())

使用 `experimental.sample_from_datasets` 将二者合并起来：

In [ ]:
resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)

In [ ]:
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())

要使用此数据集，您需要每个周期的步骤数。

在这种情况下，“周期”的定义就不那么明确了。假设它是遍历一次所有负样本所需的批次数量：

In [ ]:
resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)
resampled_steps_per_epoch

### 在过采样数据上进行训练

现在尝试使用重新采样后的数据集（而非使用类权重）来训练模型，对比一下这两种方法有何区别。

注：因为数据平衡是通过复制正样本实现的，所以数据集的总大小变大了，且每个周期运行的训练步骤也增加了。 

In [ ]:
resampled_model = make_model()
resampled_model.load_weights(initial_weights)

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).cache()
val_ds = val_ds.batch(BATCH_SIZE).prefetch(2) 

resampled_history = resampled_model.fit(
    resampled_ds,
    epochs=EPOCHS,
    steps_per_epoch=resampled_steps_per_epoch,
    callbacks = [early_stopping],
    validation_data=val_ds)

如果训练过程在每次梯度更新时都考虑整个数据集，那么这种过采样将与类加权基本相同。

但是，当按批次训练模型时（如您在上面所做的那样），过采样的数据将提供更加平滑的梯度信号：不在一个权重较大的批次中显示每个正样本，而是在许多具有较小权重的不同批次中分别显示。

这种更平滑的梯度信号使训练模型变得更加容易。

### 查看训练历史记录

请注意，此处的指标分布将有所不同，因为训练数据与验证和测试数据的分布完全不同。 

In [ ]:
plot_metrics(resampled_history )

### 重新训练


由于在平衡数据上训练更加容易，上面的训练过程可能很快就会过拟合。

因此，请打破周期，使 `callbacks.EarlyStopping` 能够更好地控制停止训练的时间。

In [ ]:
resampled_model = make_model()
resampled_model.load_weights(initial_weights)

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

resampled_history = resampled_model.fit(
    resampled_ds,
    # These are not real epochs
    steps_per_epoch = 20,
    epochs=10*EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_ds))

### 重新查看训练历史记录

In [ ]:
plot_metrics(resampled_history)

### 评估指标

In [ ]:
train_predictions_resampled = resampled_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_resampled = resampled_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
resampled_results = resampled_model.evaluate(test_features, test_labels,
                                             batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(resampled_model.metrics_names, resampled_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_resampled)

### 绘制 ROC

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')

plot_roc("Train Resampled", train_labels, train_predictions_resampled,  color=colors[2])
plot_roc("Test Resampled", test_labels, test_predictions_resampled,  color=colors[2], linestyle='--')
plt.legend(loc='lower right')

## 使用本教程解决您的问题

由于可供学习的样本过少，不平衡数据的分类是固有难题。您应该始终先从数据开始，尽可能多地收集样本，并充分考虑可能相关的特征，以便模型能够充分利用占少数的类。有时您的模型可能难以改善且无法获得想要的结果，因此请务必牢记问题的上下文，并在不同类型的错误之间进行权衡。